In [ ]:
import numpy as np
from math import log ,factorial
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,recall_score,f1_score
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
import io

In [ ]:
filename = "spambase.data"
X = []
Y = []
for line in open(filename):
    #li=line.strip()
    var = line.split(',')
    Y.append(int(var[-1]))
    X.append(map(float,var[1:len(var)-1]))

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
#to obtain features that are 0 or 1.
x=[]
for listx in X:
    for number in range(len(listx)):
        if  listx[number]> 0.0:
            listx[number] = 1.0
    x.append(listx)
print np.shape(x)

In [ ]:
#taking the number and multiplying with a random number to help obtain frequency once class segregation is complete. 
d=[]
rand=[np.random.randint(1,10) for i in range(len(x))]
for i in range(len(x)):
    d.append(x[i]*rand[i])
#print list(d)    

In [ ]:
class0 = []
class1 = []

for i in range(len(Y)):
    if Y[i] == 0:
        class0.append(d[i])
    elif Y[i]==1:
        class1.append(d[i])

In [ ]:
#while generating alpha values we set a smoothing value (epsilon = 1) and add to numerator and denominator as given in formula.
alpha0 = []
alpha0.append((np.sum(class0,axis = 0)+1)/ (len(class0)+1))
print alpha0

In [ ]:
alpha1 = []
alpha1.append((np.sum(class1,axis = 0)+1)/ (len(class1)+1))
print alpha1

In [ ]:
prior0 = 0
prior0 = float(len(class0))/float(len(x))
print prior0

In [ ]:
prior1 = 0
prior1 = float(len(class1))/float(len(x))
print prior1

In [ ]:
def nCr(n,r):
    f = factorial
    return f(n) / f(r) / f(n-r)

In [ ]:
mem1 = []
tot = np.sum(d) #Total number of words in data
u = np.array(d)
#print np.shape(u)
for i in range(len(alpha0)):
    for j in range(len(u)):
        delim=0
        for k in range(len(u[j])):
            #print alpha0[i][k]
            ncr1 = log(nCr(tot,u[j][k]))
            x1 = u[j][k]*alpha0[i][k]
            x2 = (tot-u[j][k])*(1-alpha0[i][k])
            delim = delim+ncr1+x1+x2
        delim = delim + log(prior0)
        mem1.append(delim)

In [ ]:
mem2 = []
tot = np.sum(x) #Total number of words in data
v = np.array(x)
#print np.shape(u)
for i in range(len(alpha1)):
    for j in range(len(v)):
        delim1=0
        for k in range(len(v[j])):
            #print u[i][j]
            ncr2 = log(nCr(tot,v[j][k]))
            y1 = v[j][k]*alpha1[i][k]
            y2 = (tot-v[j][k])*(1-alpha1[i][k])
            delim1 = delim1+ncr2+y1+y2
        delim1 = delim1 + log(prior1)
        mem2.append(delim1)

In [ ]:
kf = KFold(len(Y), n_folds=10, shuffle = True)
cm_fold = []

In [ ]:
for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    #to obtain features that are 0 or 1.
    x=[]
    for listx in X:
        for number in range(len(listx)):
            if  listx[number]> 0.0:
                listx[number] = 1.0
        x.append(listx)
    #print np.shape(x)
    #taking the number and multiplying with a random number to help obtain frequency once class segregation is complete. 
    d=[]
    rand=[np.random.randint(1,10) for i in range(len(x))]
    for i in range(len(x)):
        d.append(x[i]*rand[i])
    #print list(d)    
    class0 = []
    class1 = []

    for i in range(len(Y)):
        if Y[i] == 0:
            class0.append(d[i])
        elif Y[i]==1:
            class1.append(d[i])
            
    #while generating alpha values we set a smoothing value (epsilon = 1) and add to numerator and denominator as given in formula.
    alpha0 = []
    alpha0.append((np.sum(class0,axis = 0)+1)/ (len(class0)+1))
    print alpha0

    alpha1 = []
    alpha1.append((np.sum(class1,axis = 0)+1)/ (len(class1)+1))
    print alpha1

    mem1 = []
    tot = np.sum(d) #Total number of words in data
    u = np.array(d)
    #print np.shape(u)
    for i in range(len(alpha0)):
        for j in range(len(u)):
            delim=0
            for k in range(len(u[j])):
                #print alpha0[i][k]
                ncr1 = log(nCr(tot,u[j][k]))
                x1 = u[j][k]*alpha0[i][k]
                x2 = (tot-u[j][k])*(1-alpha0[i][k])
                delim = delim+ncr1+x1+x2
            delim = delim + log(prior0)
            mem1.append(delim)

    mem2 = []
    tot = np.sum(x) #Total number of words in data
    v = np.array(x)
    #print np.shape(u)
    for i in range(len(alpha1)):
        for j in range(len(v)):
            delim1=0
            for k in range(len(v[j])):
                #print u[i][j]
                ncr2 = log(nCr(tot,v[j][k]))
                y1 = v[j][k]*alpha1[i][k]
                y2 = (tot-v[j][k])*(1-alpha1[i][k])
                delim1 = delim1+ncr2+y1+y2
            delim1 = delim1 + log(prior1)
            mem2.append(delim1)
        
    predict_ts = []
        for i,j in zip(mem1,mem2):
            if i>j:
                predict_ts.append(0)
            else: predict_ts.append(1)
            
    clabels = [0,1]
    cm_test= []
    for i in clabels:
        tmp =[0]*len(clabels)
        for j in range(len(Y_test)):
            if Y_test[j] == i and Y_test[j] == predict_ts[j]:
                tmp[clabels.index(i)] += 1
            elif Y_test[j] == i and Y_test[j] != predict_ts[j]:
                tmp[clabels.index(predict_ts[j])] += 1
        cm_test.append(tmp)
    cm_fold.append(cm_test)

In [ ]:
cm_fold = np.array(cm_fold)
print cm_fold

In [ ]:
prec = []
for i in cm_fold:
    i = np.array(i)
    prec_tmp=[]
    try:
        prec_tmp.append(float(i[0,0])/np.sum(i,axis=0)[0])
    except ZeroDivisionError:
        prec_tmp.append(0)
    try:
        prec_tmp.append(float(i[1,1])/np.sum(i,axis=0)[1])
    except ZeroDivisionError:
        prec_tmp.append(0)   
    prec.append(prec_tmp) 
    
print prec
np.mean(prec,axis=0)

In [ ]:
rec = []
for i in cm_fold:
    i = np.array(i)
    rec_tmp=[]
    try:
        rec_tmp.append(float(i[0,0])/np.sum(i,axis=1)[0])
    except ZeroDivisionError:
        rec_tmp.append(0)
    try:
        rec_tmp.append(float(i[1,1])/np.sum(i,axis=1)[1])
    except ZeroDivisionError:
        rec_tmp.append(0) 
    rec.append(rec_tmp)     
    
print rec
np.mean(rec,axis=0)

In [ ]:
F1 = [] #fmeasure
for i in range(len(prec)):
    tmp = []
    try:
        varr0 = 2*((prec[i][0]*rec[i][0])/(prec[i][0]+rec[i][0]))
        #print varr0
        tmp.append(varr0)
    except ZeroDivisionError:
        tmp.append(0)
    try:
        varr1 = 2*((prec[i][1]*rec[i][1])/(prec[i][1]+rec[i][1]))
        #print varr1
        tmp.append(varr1)
    except ZeroDivisionError:
        tmp.append(0)
    F1.append(tmp)
    
print F1
np.mean(F1,axis = 0)

In [ ]:
acc = []
for i in range(len(cm_fold)):
    acc1 = np.trace(cm_fold[i])*1.0/np.sum(cm_fold[i])
    acc.append(acc1)
print np.mean(acc)